In [1]:
import numpy as np
import pandas as pd

In [2]:
import yfinance as yf

df = yf.download('AMD', start="2015-01-01")
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-02,2.670000,2.670000,2.670000,2.670000,2.670000,0
2015-01-05,2.670000,2.700000,2.640000,2.660000,2.660000,8878200
2015-01-06,2.650000,2.660000,2.550000,2.630000,2.630000,13912500
2015-01-07,2.630000,2.650000,2.540000,2.580000,2.580000,12377600
2015-01-08,2.590000,2.650000,2.560000,2.610000,2.610000,11136600
...,...,...,...,...,...,...
2024-04-05,168.100006,172.690002,165.580002,170.419998,170.419998,66025200
2024-04-08,168.199997,171.660004,166.820007,169.899994,169.899994,43997500
2024-04-09,170.210007,171.600006,167.289993,170.779999,170.779999,42928000


In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_df = scaler.fit_transform(df[['Close']].values)
scaled_df

array([[-1.0333809 ],
       [-1.03359333],
       [-1.0342306 ],
       ...,
       [ 2.53769698],
       [ 2.46037427],
       [ 2.50689314]])

In [4]:
# Define parameters
input_length = 100  # Number of time steps to look back
output_length = 20  # Number of time steps to predict ahead
n_features = 1  # Number of input features (only 'Close' price)
n_units = 64  # Number of units in GRU layers
dropout_rate = 0.2  # Dropout rate

In [5]:
# Create time series batches
X, Y = [], []
for i in range(len(scaled_df) - input_length - output_length + 1):
    X.append(scaled_df[i:i+input_length])
    Y.append(scaled_df[i+input_length:i+input_length+output_length])

X = np.array(X)
Y = np.array(Y)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam

# Define LSTM model
model = Sequential([
    LSTM(units=n_units, input_shape=(input_length, n_features), return_sequences=True),
    Dropout(dropout_rate),
    LSTM(units=n_units),
    Dropout(dropout_rate),
    Dense(units=output_length)
])

# Compile model
model.compile(optimizer=Adam(), loss='mse')

In [7]:
from keras.callbacks import EarlyStopping

callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X, Y, epochs=100, validation_split=0.2, callbacks=[callback])

Epoch 1/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 6s 58ms/step - loss: 0.3514 - val_loss: 0.2386
Epoch 2/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - loss: 0.0482 - val_loss: 0.1298
Epoch 3/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - loss: 0.0407 - val_loss: 0.1363
Epoch 4/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - loss: 0.0338 - val_loss: 0.1330
Epoch 5/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - loss: 0.0338 - val_loss: 0.1416
Epoch 6/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - loss: 0.0300 - val_loss: 0.0967
Epoch 7/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - loss: 0.0309 - val_loss: 0.1147
Epoch 8/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - loss: 0.0279 - val_loss: 0.1204
Epoch 9/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - loss: 0.0286 - val_loss: 0.1183
Epoch 10/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - loss: 0.0260 - val_loss: 0.1443
Epoch 11/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - loss: 0.0254 - val_loss: 0.0998
Epoch 12/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step

In [8]:
# Predictions
def predict_all(model, data, input_length, output_length):
    predictions = []
    for i in range(len(data) - input_length + 1):
        input_data = data[i:i+input_length].reshape(1, input_length, n_features)
        pred = model.predict(input_data).reshape(output_length)
        predictions.append(pred)
    return np.array(predictions)

scaled_preds = predict_all(model, scaled_df, input_length, output_length)
preds = scaler.inverse_transform(scaled_preds)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━

In [11]:
import plotly.graph_objects as go

candles_back = 300
lpadding = 100
rpadding = 100

rlim = rpadding-candles_back if candles_back > rpadding else None

df_display = df.iloc[-lpadding-output_length-candles_back:rlim]

fig = go.Figure(data=[
    go.Candlestick(
        x=df_display.index,
        open=df_display['Open'],
        high=df_display['High'],
        low=df_display['Low'],
        close=df_display['Close']
    )
])

fig.add_trace(go.Scatter(
    x=df_display.index[lpadding:lpadding+output_length],
    y=preds[-output_length-candles_back-1],
    line_shape='hv',
    mode='lines'
))

fig.add_vrect(
    x0=df_display.index[lpadding],
    x1=df_display.index[lpadding+output_length-1],
    annotation_text="FORECAST",
    annotation_position="top left",
    fillcolor="green",
    opacity=0.2,
    line_width=0
)

fig.show()